In [7]:
# Notebook to read in the nexus files and only save the useful stuff to slim the files down
import os
import sys
import tables as tb
import numpy  as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import re

In [8]:
# Loading in the file
# input_file = "../data/nexus/0nuBB2.next.h5"
# input_file = "../data/nexus/0nuBB2_10bar.next.h5"
input_file = "/Users/mistryk2/Packages/nexus/0nuBB2.next.h5"

h5 = tb.open_file(input_file, 'r')
## Configuration
config = pd.read_hdf(input_file, 'MC/configuration')
display(config)

energy = ""

pressure = config[ config["param_key"] == "/Geometry/XeSphere/pressure"].iloc[0]['param_value']
pressure = [float(num) if '.' in num else int(num) for num in pressure.split() if num.replace('.', '', 1).isdigit()]
pressure = f"{int(pressure[0])}_bar"


eventtype = config[ config["param_key"] == "/nexus/RegisterGenerator"].iloc[0]['param_value']

if (eventtype == "Decay0Interface"):
    eventtype = "bb"
    energy = "2.5_MeV"
else:
    eventtype = "singleeminus"

    # Then also get the energy
    energy = config[ config["param_key"] == "/Generator/SingleParticle/min_energy"].iloc[0]['param_value']
    energy = [float(num) if '.' in num else int(num) for num in energy.split() if num.replace('.', '', 1).isdigit()]
    energy = f"{energy[0]}_MeV"
    

print("Pressure:", pressure) 
print("Energy:", energy) 
print(eventtype)


,param_key,param_value
0,event_type,background
1,num_events,5000
2,saved_events,5000
3,interacting_events,5000
4,/PhysicsList/RegisterPhysics,G4EmPenelopePhysics
5,/PhysicsList/RegisterPhysics,G4DecayPhysics
6,/PhysicsList/RegisterPhysics,G4RadioactiveDecayPhysics
7,/PhysicsList/RegisterPhysics,NexusPhysics
8,/PhysicsList/RegisterPhysics,G4StepLimiterPhysics
9,/nexus/RegisterGeometry,XeSphere


Pressure: 1_bar
Energy: 2.5_MeV
bb


In [9]:
## Particles
parts = pd.read_hdf(input_file, 'MC/particles')
print(parts.columns.tolist())
parts = parts.drop(columns=['initial_volume', 'final_volume',
                            'initial_momentum_x', 'initial_momentum_y', 'initial_momentum_z',
                            'final_momentum_x', 'final_momentum_y', 'final_momentum_z'])
display(parts)

['event_id', 'particle_id', 'particle_name', 'primary', 'mother_id', 'initial_x', 'initial_y', 'initial_z', 'initial_t', 'final_x', 'final_y', 'final_z', 'final_t', 'initial_volume', 'final_volume', 'initial_momentum_x', 'initial_momentum_y', 'initial_momentum_z', 'final_momentum_x', 'final_momentum_y', 'final_momentum_z', 'kin_energy', 'length', 'creator_proc', 'final_proc']


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,final_y,final_z,final_t,kin_energy,length,creator_proc,final_proc
0,0,2,e-,1,0,0.000000,0.000000,0.000000,0.000000,196.420898,-526.592285,214.542847,6.521577,1.073863,1649.938232,none,eIoni
1,0,764,e-,0,2,196.336823,-526.620483,214.551498,6.517659,196.336838,-526.620483,214.551514,6.517664,0.000021,0.000016,eIoni,msc
2,0,763,e-,0,2,196.336823,-526.620483,214.551498,6.517659,196.336182,-526.620728,214.550720,6.518050,0.000021,0.001843,eIoni,eIoni
3,0,762,e-,0,2,196.336823,-526.620483,214.551498,6.517659,196.335922,-526.620972,214.551300,6.518050,0.000021,0.001155,eIoni,msc
4,0,761,e-,0,2,196.336823,-526.620483,214.551498,6.517659,196.336853,-526.620483,214.551483,6.517666,0.000046,0.000030,eIoni,msc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9724371,4999,945,e-,0,1,-1.311255,-0.014726,0.642534,0.005127,-1.311068,-0.015278,0.643076,0.005323,0.000047,0.000795,eIoni,msc
9724372,4999,944,e-,0,1,-1.311255,-0.014726,0.642534,0.005127,-1.310730,-0.014366,0.642518,0.005364,0.000021,0.000637,eIoni,msc
9724373,4999,943,e-,0,1,-1.311255,-0.014726,0.642534,0.005127,-1.311359,-0.015001,0.642096,0.005321,0.000021,0.000527,eIoni,msc
9724374,4999,942,e-,0,1,-1.311255,-0.014726,0.642534,0.005127,-1.311344,-0.014977,0.640916,0.005569,0.000449,0.020073,eIoni,eIoni


In [10]:
## True hits (deposited energy)
hits = pd.read_hdf(input_file, 'MC/hits')
hits = hits.drop(columns=['label'])

display(hits)

,event_id,x,y,z,time,energy,particle_id,hit_id
0,0,-0.000311,-0.010224,-0.004853,0.000040,0.000003,2,0
1,0,-0.014672,-0.549488,-0.267121,0.002154,0.000142,2,1
2,0,-0.007215,-1.356258,-0.665449,0.005325,0.000094,2,2
3,0,-0.007987,-1.712557,-0.847058,0.006734,0.000041,2,3
4,0,-0.010580,-1.801371,-0.892898,0.007086,0.000015,2,4
...,...,...,...,...,...,...,...,...
72621959,4999,-1.311359,-0.015001,0.642096,0.005321,0.000021,943,0
72621960,4999,-1.311288,-0.015110,0.641656,0.005203,0.000028,942,0
72621961,4999,-1.311367,-0.016043,0.639524,0.005395,0.000132,942,1
72621962,4999,-1.311344,-0.014977,0.640916,0.005569,0.000289,942,2


In [11]:
# Open the HDF5 file in write mode
with pd.HDFStore(f"../data/nexus/1bar/models/xesphere_{eventtype}_{pressure}_{energy}_penelope.h5", mode='w', complevel=5, complib='zlib') as store:
    # Write each DataFrame to the file with a unique key
    store.put('config', config, format='table')
    store.put('parts',parts, format='table')
    store.put('hits',hits, format='table')

In [12]:
# load back in to check

hits2 = pd.read_hdf(f"../data/nexus/1bar/models/xesphere_{eventtype}_{pressure}_{energy}_penelope.h5", 'hits')
display(hits2)

,event_id,x,y,z,time,energy,particle_id,hit_id
0,0,-0.000311,-0.010224,-0.004853,0.000040,0.000003,2,0
1,0,-0.014672,-0.549488,-0.267121,0.002154,0.000142,2,1
2,0,-0.007215,-1.356258,-0.665449,0.005325,0.000094,2,2
3,0,-0.007987,-1.712557,-0.847058,0.006734,0.000041,2,3
4,0,-0.010580,-1.801371,-0.892898,0.007086,0.000015,2,4
...,...,...,...,...,...,...,...,...
72621959,4999,-1.311359,-0.015001,0.642096,0.005321,0.000021,943,0
72621960,4999,-1.311288,-0.015110,0.641656,0.005203,0.000028,942,0
72621961,4999,-1.311367,-0.016043,0.639524,0.005395,0.000132,942,1
72621962,4999,-1.311344,-0.014977,0.640916,0.005569,0.000289,942,2
